In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [3]:
%%sql
drop table if exists RacingResults, HorseNames;
CREATE TABLE RacingResults
(track_id CHAR(3) NOT NULL,
 race_date DATE NOT NULL,
 race_nbr INTEGER NOT NULL,
 win_name CHAR(30) NOT NULL,
 place_name CHAR(30) NOT NULL,
 show_name CHAR(30) NOT NULL,
    PRIMARY KEY (track_id, race_date, race_nbr));

CREATE TABLE HorseNames
(horse CHAR(30) NOT NULL PRIMARY KEY);

INSERT INTO RacingResults VALUES(1, '2007-05-01', 1, 'A', 'B', 'C');
INSERT INTO RacingResults VALUES(1, '2007-05-01', 2, 'E', 'F', 'P');
INSERT INTO RacingResults VALUES(1, '2007-05-02', 1, 'B', 'C', 'A');
INSERT INTO RacingResults VALUES(2, '2007-05-02', 1, 'O', 'P', 'Q');
INSERT INTO RacingResults VALUES(2, '2007-05-02', 2, 'A', 'P', 'Q');

INSERT INTO HorseNames VALUES('A');
INSERT INTO HorseNames VALUES('B');
INSERT INTO HorseNames VALUES('C');
INSERT INTO HorseNames VALUES('D');
INSERT INTO HorseNames VALUES('E');
INSERT INTO HorseNames VALUES('F');
INSERT INTO HorseNames VALUES('O');
INSERT INTO HorseNames VALUES('P');
INSERT INTO HorseNames VALUES('Q');

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [ ]:
## やりたいこと
+ 各馬について1位、2位, 3位の回数をそれぞれ集計する
  + 1~3位の合計入賞回数ももとめるには?

In [10]:
%%sql
-- selectの中にcaseをいれる
select h.horse,
    count(case when r.win_name = h.horse then 1 else null end) as first,
    count(case when r.place_name = h.horse then 1 else null end) as second,
    count(case when r.show_name = h.horse then 1 else null end) as third
from RacingResults as r, HorseNames as h
group by h.horse
order by h.horse

*  postgresql://padawan:***@db:5432/dsdojo_db
9 rows affected.


horse,first,second,third
A,2,0,1
B,1,1,0
C,0,1,1
D,0,0,0
E,1,0,0
F,0,1,0
O,1,0,0
P,0,2,1
Q,0,0,2


In [6]:
%%sql
-- スカラサブクエリで
select h.horse, (
    select count(*)
    from RacingResults as r
    where h.horse = r.win_name
), (
    select count(*)
    from RacingResults as r
    where h.horse = r.place_name
), (
    select count(*)
    from RacingResults as r
    where h.horse = r.show_name
)
from HorseNames as h

*  postgresql://padawan:***@db:5432/dsdojo_db
9 rows affected.


horse,count,count_1,count_2
A,2,0,1
B,1,1,0
C,0,1,1
D,0,0,0
E,1,0,0
F,0,1,0
O,1,0,0
P,0,2,1
Q,0,0,2


In [5]:
%%sql
-- union allで列持ち -> 行持ちにしてから、集計
select h.horse,
    count(case when rank = 1 then 1 else null end) as first,
    count(case when rank = 2 then 1 else null end) as second,
    count(case when rank = 3 then 1 else null end) as third
from (
    select race_date, win_name, 1
    from RacingResults
    union all
    select race_date, place_name, 2
    from RacingResults
    union all
    select race_date, show_name, 3
    from RacingResults
) as tmp(race_date, horse, rank)
right join HorseNames as h
on tmp.horse = h.horse
group by h.horse
order by h.horse

*  postgresql://padawan:***@db:5432/dsdojo_db
9 rows affected.


horse,first,second,third
A,2,0,1
B,1,1,0
C,0,1,1
D,0,0,0
E,1,0,0
F,0,1,0
O,1,0,0
P,0,2,1
Q,0,0,2


In [7]:
%%sql
-- ちなみに1~3位の合計入賞回数ももとめる場合、さらに簡単にできる:
select h.horse, count(*)
from RacingResults as r, HorseNames as h
where h.horse in (r.win_name, r.place_name, r.show_name)
group by h.horse
order by h.horse

*  postgresql://padawan:***@db:5432/dsdojo_db
8 rows affected.


horse,count
A,3
B,2
C,2
E,1
F,1
O,1
P,3
Q,2
